In [1]:
!python -V


Python 3.6.13 :: Anaconda, Inc.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from ipywidgets import interact, widgets

# 加载FITS数据
fits_name = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
with fits.open(fits_name) as hdul:
    data = hdul[0].data
    header = hdul[0].header
wcs = WCS(header)

# 计算速度值
velocities = (np.arange(header['NAXIS3']) - (header['CRPIX3']-1)) * header['CDELT3'] / 1000.0 + header['CRVAL3'] / 1000.0  # km/s

# 自定义函数来同时更新XY图、VY图和XV图
def update_plots(x_channel, y_channel):
    ra_line = wcs.all_pix2world([[x_channel, 0, 0]], 0)[0][0]
    dec_line = wcs.all_pix2world([[0, y_channel, 0]], 0)[0][1]
    fig, axs = plt.subplots(2, 2, figsize=(14, 14))

    # 确保所有的图像都是正方形
    for ax in axs.flat:
        ax.set_aspect('equal', 'box')

    # XY图
    xy_data = np.sum(data, axis=0)  # 对所有速度通道求和
    im1 = axs[0, 0].imshow(xy_data, origin='lower', cmap='nipy_spectral', aspect='auto')
    fig.colorbar(im1, ax=axs[0, 0], fraction=0.046, pad=0.04)
    axs[0, 0].axvline(x=x_channel, color='white')
    axs[0, 0].axhline(y=y_channel, color='white')
    axs[0, 0].set_title('XY Map')

    # VY图
    vy_data = data[:, :, x_channel].T
    im2 = axs[0, 1].imshow(vy_data, origin='lower', cmap='nipy_spectral', aspect='auto',
                           extent=[velocities[0], velocities[-1], 0, data.shape[1]])
    fig.colorbar(im2, ax=axs[0, 1], fraction=0.046, pad=0.04)
    axs[0, 1].set_title('VY Map')

    # XV图
    xv_data = data[:, y_channel, :]
    im3 = axs[1, 0].imshow(xv_data, origin='lower', cmap='nipy_spectral', aspect='auto',
                           extent=[0, data.shape[2], velocities[0], velocities[-1]])
    fig.colorbar(im3, ax=axs[1, 0], fraction=0.046, pad=0.04)
    axs[1, 0].set_title('XV Map')

    # 删除右下角的空图
    fig.delaxes(axs[1, 1])

    # 调整布局以避免重叠
    plt.tight_layout()
    plt.show()
    return f"Current RA: {ra_line:.4f} Degrees, DEC: {dec_line:.4f} Degrees"

# 创建交互式控件
interact(update_plots,
         x_channel=widgets.IntSlider(min=0, max=data.shape[2]-1, step=1, value=data.shape[2]//2, description='X channel'),
         y_channel=widgets.IntSlider(min=0, max=data.shape[1]-1, step=1, value=data.shape[1]//2, description='Y channel'))


interactive(children=(IntSlider(value=267, description='X channel', max=533), IntSlider(value=266, description…

<function __main__.update_plots(x_channel, y_channel)>